# Clasificación de imágenes (Nivel Medio)
---
<style>
      h1, h2, h3, h4, h5, h6,.imagen {
        text-align: center;
      }
 img{width: 75%; height: 75%;}
</style>

### Instalación de librerías

```python
%pip install torch torchvision matplotlib numpy tqdm scikit-learn pillow datasets

```

### Descarga de datos

```python
from datasets import load_dataset


datasetdict=load_dataset("mnist")
datasetdict

```

### Separación de datos


```python
train_dataset=datasetdict["train"]
test_dataset=datasetdict["test"]
train_dataset
```

### Visualización de datos

#### Imagen/etiqueta

```python

train_dataset[65]["image"]

#------------------------
from PIL import Image
import matplotlib.pyplot as plt

def show_image(image, size=(8, 8)):
    
    fig, ax = plt.subplots(figsize=size)
    ax.grid(False)
   
    
    ax.axis('off')
    ax.imshow(image)
    
    plt.show()
#------------------------

index=5 
show_image(train_dataset[index]["image"], size=(2, 2))
train_dataset[index]["label"]
    
```

#### Mostrar pixeles
 

```python

import numpy as np
import pandas as pd
#------------------------

def Mostrar_pixeles(image):
    img=image.convert('L')
    np_array=np.array(img)
    df=pd.DataFrame(np_array)
    
    return df.style.set_properties(**{'font-size':'6pt'}).background_gradient('Greys')

#------------------------
Mostrar_pixeles(train_dataset[10]["image"])

```



### Modelo

#### Dispositivo

```python

dispotivo=torch.device("cuda" if torch.cuda.is_available() else "cpu")
dispotivo

```

 
#### Modelo

```python

import torch


#------------------------
x,y=train_dataset[0]["image"].size
caracteristicas=x*y
 
parametrosocultos=64

#------------------------
pesos1=torch.randn(caracteristicas,parametrosocultos).to(dispotivo) .requires_grad_()
sesgo1=torch.zeros(parametrosocultos).to(dispotivo).requires_grad_()

pesos2=torch.randn(parametrosocultos,parametrosocultos).to(dispotivo).requires_grad_()
sesgo2=torch.zeros(parametrosocultos).to(dispotivo).requires_grad_()

pesos3=torch.randn(parametrosocultos,10).to(dispotivo).requires_grad_()
sesgo3=torch.zeros(10).to(dispotivo).requires_grad_()

```

#### Preparar datos

```python
from torchvision.transforms import ToTensor, Normalize

#------------------------

def pil_to_flat_tensor(images):
    # Convertir la lista de imágenes PIL a una lista de tensores
    tensors = [ToTensor()(image) for image in images]
    # Normalizar los tensores
    transform = Normalize(mean=[0.5], std=[0.5])
    tensors = [transform(tensor) for tensor in tensors]
    # Apilar los tensores en una matriz
    tensor_matrix = torch.stack(tensors)
    # Aplanar la matriz de tensores
    flat_tensor_matrix = tensor_matrix.view(len(images), -1)
    return flat_tensor_matrix


#------------------------



X=pil_to_flat_tensor(X)
X=X.to(dispotivo)
y=torch.tensor(y).to(dispotivo)
X[:,300]
```


#### Función de pérdida

```python
from torch.nn.functional import cross_entropy


```

#### Forward pass



```python

resultado=X@pesos1
resultado=resultado+sesgo1
resultado=resultado.clamp_min(0.)

resultado=resultado@pesos2+sesgo2
resultado=resultado.clamp_min(0.)

resultado=resultado@pesos3+sesgo3

funcion_perdida=cross_entropy(resultado,y )

prediccion=torch.argmax(resultado,dim=1)==y.long()
prediccion.float().mean()




```

#### Backward pass

```python

funcion_perdida.backward()
pesos1.grad
#------------------------

with torch.no_grad():
    
    pesos1-=pesos1.grad*1e-3
    
    
    pesos2-=pesos2.grad*1e-3
    pesos3-=pesos3.grad*1e-3
    
    sesgo1-=sesgo1.grad*1e-3
    
    sesgo2-=sesgo2.grad*1e-3
    
    sesgo3-=sesgo3.grad*1e-3
    
    pesos1.grad.zero_()
    
    pesos2.grad.zero_()
    
    pesos3.grad.zero_()
    
    sesgo1.grad.zero_()
    
    sesgo2.grad.zero_()
    
    sesgo3.grad.zero_()

```

#### Metricas

```python

with torch.no_grad():
    resultado=X@pesos1+sesgo1
    resultado=resultado.clamp_min(0.)
    resultado=resultado@pesos2+sesgo2
    resultado=resultado.clamp_min(0.)
    resultado=resultado@pesos3+sesgo3
    y_pred=torch.argmax(resultado,dim=1)
    precision=(y_pred==y).sum()/len(y)
    print(f"Prediccion: {y_pred==y}")
    print(f"Precision: {precision:.4f}")

```

#### Entrenamiento

##### Hiperparámetros

```python
import torch.nn as nn
from tqdm import tqdm 
#------------------------

lr=3e-2
epocas=10
batch_size = 64
loss=nn.CrossEntropyLoss()

#------------------------

```

##### Bucle de entrenamiento

```python

for epoca in range(epocas):
    train_dataset = train_dataset.shuffle()
    perdida=0
    error=0
    acierto=0
    ab=0
    for i in tqdm(range(0, len(train_dataset), batch_size)):
        batch= train_dataset[i:i+batch_size]
        X, y = batch["image"], batch["label"]
        X = pil_to_flat_tensor(X)
        X = X.to(dispotivo)
        y = torch.Tensor(y).long().to(dispotivo)
        
        resultado = X@pesos1+ sesgo1
        resultado = resultado.clamp_min(0.)
        resultado = resultado@pesos2+ sesgo2
        resultado = resultado.clamp_min(0.)
        resultado = resultado@pesos3+ sesgo3
        
        loss_value = loss(resultado, y)
        perdida += loss_value.item()
        acc = torch.argmax(resultado, dim=1) == y
        acc = acc.float().mean()
        acierto += acc
        
        error_rate = 1 - acc
        error += error_rate
        
        loss_value.backward()
        
        with torch.no_grad():
            pesos1 -= pesos1.grad * lr
            pesos2 -= pesos2.grad * lr
            pesos3 -= pesos3.grad * lr
            sesgo1 -= sesgo1.grad * lr
            sesgo2 -= sesgo2.grad * lr
            sesgo3 -= sesgo3.grad * lr
            pesos1.grad.zero_()
            pesos2.grad.zero_()
            pesos3.grad.zero_()
            sesgo1.grad.zero_()
            sesgo2.grad.zero_()
            sesgo2.grad.zero_()
            sesgo3.grad.zero_()
        ab+=1
            
    print(f"Entrenamiento> Epoca:{epoca+1}/{epocas}  ,Perdida: {perdida / ab:.4f}, Error: {error /ab:.4f}, Acierto: {acierto / ab:.4f}")
    perdida=0
    error=0
    acierto=0
    ab=0
    with torch.no_grad():
        for i in tqdm(range(0, len(test_dataset), batch_size*2)):
            batch = test_dataset[i:i+batch_size*2]
            X, y = batch["image"], batch["label"]
            X = pil_to_flat_tensor(X)
            X = X.to(dispotivo)
            y = torch.Tensor(y).long().to(dispotivo)
            
            resultado = X@pesos1+ sesgo1
            resultado = resultado.clamp_min(0.)
            resultado = resultado@pesos2+ sesgo2
            resultado = resultado.clamp_min(0.)
            resultado = resultado@pesos3+ sesgo3
            
            loss_value = loss(resultado, y)
            perdida += loss_value.item()
            acc = torch.argmax(resultado, dim=1) == y
            acc = acc.float().mean()
            acierto += acc.item()
            
            error_rate = 1 - acc
            error += error_rate.item()
            ab+=1
            
    print(f"Validación> Epoca:{epoca+1}/{epocas}  ,Perdida: {perdida / ab:.4f}, Error: {error / ab:.4f}, Acierto: {acierto / ab:.4f}")

```

##### Predicción

```python
test=test_dataset.shuffle()

X,y=test[:64]["image"],test[:64]["label"]
with torch.no_grad():
    X=pil_to_flat_tensor(X)
    X=X.to(dispotivo)
    y=torch.Tensor(y).long().to(dispotivo)
    
    resultado=X@pesos1+sesgo1
    resultado=resultado.clamp_min(0.)
    resultado=resultado@pesos2+sesgo2
    resultado=resultado.clamp_min(0.)
    resultado=resultado@pesos3+sesgo3
    resultado=torch.argmax(resultado,dim=1)==y
     
    
    print(resultado.float().mean())

```

## Clasificación de imágenes (Redes Neuronales Convolucionales)

### Instalación de librerías

```python

%pip install torch torchvision matplotlib numpy tqdm scikit-learn pillow datasets torcheval
    
```

###  Librerías y Datos

```python
import torch
import torchvision
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import torcheval.metrics.functional as eval

#------------------------
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),torchvision.transforms.Normalize((0.5,), (0.5,))])  

 
train_dataset = torchvision.datasets.MNIST(root='../tmp/dataset', train=True, download=True, transform=transform)

test_dataset = torchvision.datasets.MNIST(root='../tmp/dataset', train=False, download=True, transform=transform)

#------------------------
bach_size=512

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bach_size,num_workers=4, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bach_size*2,num_workers=4, shuffle=False)


#------------------------
example_data, example_targets= next(iter(train_loader))

print(example_data.shape)
cantidad = 20
fig = plt.figure()
for i in range(cantidad):
    plt.subplot(cantidad//4,4,i+1)
    plt.tight_layout()
    plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
    plt.title("class: {}".format(example_targets[i]))
    plt.xticks([])
    plt.yticks([])
fig.show()



```



### Modelo


```python
class ConvNet(nn.Module):
    def __init__(self,input_size,num_classes):
        super(ConvNet, self).__init__()
        self.conv1=nn.Conv2d(
            in_channels=input_size,
            out_channels=64,
            kernel_size=3,
            stride=1,
            padding=1           
                   )  
        
        self.BN1=nn.BatchNorm2d(64)
        self.relu=nn.ReLU()        
        self.pool=nn.AdaptiveAvgPool2d(1)        
        self.flatten=nn.Flatten()        
        self.linear=nn.Sequential(
            nn.Linear(64, 512), 
            nn.ReLU(),
            nn.Linear(512, num_classes))
        
    def forward(self,x):
        x=self.conv1(x)
        x=self.BN1(x)
        x=self.relu(x)
        x=self.pool(x)
        x=self.flatten(x)
        x=self.linear(x)
        return x        
        
#--------------------------

class ConvNet2(nn.Module):
    def __init__(self,input_size,num_classes):
        super(ConvNet2, self).__init__()
        
        self.conv1=nn.Sequential(
             nn.Conv2d(in_channels=input_size, out_channels=32, kernel_size=7, stride=1, padding=2, bias=False),
             nn.BatchNorm2d(32),
             nn.ReLU(),
             
          nn.Conv2d(in_channels=32, out_channels=64, kernel_size=7, stride=1, padding=2, bias=False),
                nn.BatchNorm2d(64),
                nn.ReLU(),
                nn.MaxPool2d(2,2),
        )
     
        self.conv2=nn.Sequential(   
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=5, stride=2, padding=2, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            
            nn.Conv2d(in_channels=64, out_channels=96, kernel_size=5, stride=2, padding=2, bias=False),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
        )
     
        self.conv3=nn.Sequential(
            nn.Conv2d(in_channels=96, out_channels=224, kernel_size=3, stride=1, padding=2, bias=False),
            nn.BatchNorm2d(224),
            nn.ReLU(),
            
            nn.Conv2d(in_channels=224, out_channels=224, kernel_size=3, stride=2, padding=2, bias=False),
            nn.BatchNorm2d(224),
            nn.ReLU(),
           
        ) 
        
        self.aplanar=nn.Sequential(
         nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
        
        )
        self.linear=nn.Sequential(
            nn.Linear(224, 1024),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(1024, num_classes),
            
        )
    
    def forward(self, x):
        x=self.conv1(x)
        x=self.conv2(x)
        x=self.conv3(x) 
        x=self.aplanar(x)
        x=self.linear(x)
        return x
#--------------------------
#modelo=ConvNet(1,10)
modelo=ConvNet2(1,10)
modelo(example_data)

```



### Hipérparametros

```python
dispositivo = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(dispositivo)
!nvidia-smi -L
#--------------------------
modelo=ConvNet2(1,10).to(dispositivo)
modelo

#--------------------------
#limpiar memoria 
import gc
torch.cuda.empty_cache()
gc.collect()

#--------------------------

lr=3e-3
epocas=5

optimizador=torch.optim.Adam(modelo.parameters(),lr=lr)

criterio=nn.CrossEntropyLoss()

#--------------------------
perdida_entrenamiento=[]
perdida_validacion=[]
error_rate_entrenamiento=[]
error_rate_validacion=[]
acuracy_entrenamiento=[]
acuracy_validacion=[]

#--------------------------
 
```

### Loop de entrenamiento

```python

for epoca in range(epocas):
    perdida=0
    acuracy=0
    error_rate=0
     
    modelo.train()
    
    for imagenes,etiquetas in tqdm(train_loader):
        imagenes,etiquetas=imagenes.to(dispositivo),etiquetas.to(dispositivo)
        optimizador.zero_grad()
        salida=modelo(imagenes)
        loss=criterio(salida,etiquetas)
        loss.backward()
        optimizador.step()
        
        perdida+=loss.item()
        tmpacuracy=eval.multiclass_accuracy(salida,etiquetas).item() 
        acuracy+=tmpacuracy
        
        error_rate+=1-tmpacuracy
    
    
    perdida=perdida/len(train_loader)
    acuracy=acuracy/len(train_loader)
    error_rate=error_rate/len(train_loader)    
    perdida_entrenamiento.append(perdida )
    acuracy_entrenamiento.append(acuracy )
    error_rate_entrenamiento.append(error_rate  )
    
    perdida=0
    acuracy=0
    error_rate=0
    
    modelo.eval()
    with torch.no_grad():
        for imagenes,etiquetas in tqdm(test_loader):
            imagenes,etiquetas=imagenes.to(dispositivo),etiquetas.to(dispositivo)
            
            salida=modelo(imagenes)
            loss=criterio(salida,etiquetas)
            
            perdida+=loss.item()
            
            tmpacuracy=eval.multiclass_accuracy(salida,etiquetas).item() 
            acuracy+=tmpacuracy
        
            error_rate+=1-tmpacuracy
            
    perdida=perdida/len(test_loader)
    acuracy=acuracy/len(test_loader)
    error_rate=error_rate/len(test_loader)
    perdida_validacion.append(perdida )
    acuracy_validacion.append(acuracy)
    error_rate_validacion.append(error_rate )
    
    print(f'Epoca: {epoca+1}/{epocas} Perdida entrenamiento: {perdida_entrenamiento[-1]:.4f} Perdida validacion: {perdida_validacion[-1]:.4f} Acuracy entrenamiento: {acuracy_entrenamiento[-1]*100:.4f} Acuracy validacion: {acuracy_validacion[-1]*100:.4f} Error rate entrenamiento: {error_rate_entrenamiento[-1]*100:.4f} Error rate validacion: {error_rate_validacion[-1]*100:.4f}') 
    
    
```

#### Visualización de resultados

```python

# Crear figura con subplots
fig, axs = plt.subplots(3, 1, figsize=(8, 12))

# Graficar pérdida
axs[0].plot(perdida_entrenamiento, label='Entrenamiento')
axs[0].plot(perdida_validacion, label='Validación')
axs[0].set_title('Pérdida')
axs[0].set_xlabel('Época')
axs[0].set_ylabel('Valor')
axs[0].legend()

# Graficar tasa de error
axs[1].plot(error_rate_entrenamiento, label='Entrenamiento')
axs[1].plot(error_rate_validacion, label='Validación')
axs[1].set_title('Tasa de error')
axs[1].set_xlabel('Época')
axs[1].set_ylabel('Porcentaje')
axs[1].legend()

# Graficar precisión
axs[2].plot(acuracy_entrenamiento, label='Entrenamiento')
axs[2].plot(acuracy_validacion, label='Validación')
axs[2].set_title('Precisión')
axs[2].set_xlabel('Época')
axs[2].set_ylabel('Porcentaje')
axs[2].legend()


# Ajustar el espacio entre los subplots
plt.subplots_adjust(hspace=0.5)

# Mostrar figura
plt.show()

```